In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

spark = SparkSession.builder \
.master("local") \
.appName("CS in HT") \
.getOrCreate()

In [2]:
paid = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Paid_Services_sample.csv")
paid.printSchema()
paid.registerTempTable('p')
paid.show(2)

root
 |-- Clearing_Period: integer (nullable = true)
 |-- OIB: string (nullable = true)
 |-- Document_Number: long (nullable = true)
 |-- Document_Type: string (nullable = true)
 |-- Days_between_Due_and_Clearing_Date: integer (nullable = true)
 |-- Amount: double (nullable = true)
 |-- IS_Cleared_with_Payment: integer (nullable = true)
 |-- Cleared_with_Document_Number: long (nullable = true)
 |-- Cleared_with_Document_Type: string (nullable = true)

+---------------+-----------+---------------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+
|Clearing_Period|        OIB|Document_Number|Document_Type|Days_between_Due_and_Clearing_Date|Amount|IS_Cleared_with_Payment|Cleared_with_Document_Number|Cleared_with_Document_Type|
+---------------+-----------+---------------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------

In [3]:

paid_df = spark.sql("select * from p where Document_Type not in ('CF','EN','IC','NP','PO','SR')")
paid_df.show()

+---------------+-----------+---------------+-------------+----------------------------------+-------+-----------------------+----------------------------+--------------------------+
|Clearing_Period|        OIB|Document_Number|Document_Type|Days_between_Due_and_Clearing_Date| Amount|IS_Cleared_with_Payment|Cleared_with_Document_Number|Cleared_with_Document_Type|
+---------------+-----------+---------------+-------------+----------------------------------+-------+-----------------------+----------------------------+--------------------------+
|         201707|50270059011|    20331874213|           RK|                                -3|  366.5|                      1|                 44624209266|                        UZ|
|         201708|23386350591|    20179457076|           RB|                                 3|  80.08|                      1|                 44823969505|                        UN|
|         201710|42020201434|    19273068835|           IN|                          

In [7]:
df2=paid_df.withColumn("Interval",f.expr("case when Days_between_Due_and_Clearing_Date<=0 then 'ON_TIME'\
                                        when Days_between_Due_and_Clearing_Date>=44 and Days_between_Due_and_Clearing_Date<=63 then 4463\
                                        when Days_between_Due_and_Clearing_Date>=64 and Days_between_Due_and_Clearing_Date<=93 then 6493\
                                        when Days_between_Due_and_Clearing_Date>=94  then 94\
                                        when Days_between_Due_and_Clearing_Date>=1 and Days_between_Due_and_Clearing_Date<=43 then 'remainder' end"))

df2.show()
df2.registerTempTable('v')

+---------------+-----------+---------------+-------------+----------------------------------+-------+-----------------------+----------------------------+--------------------------+---------+
|Clearing_Period|        OIB|Document_Number|Document_Type|Days_between_Due_and_Clearing_Date| Amount|IS_Cleared_with_Payment|Cleared_with_Document_Number|Cleared_with_Document_Type| Interval|
+---------------+-----------+---------------+-------------+----------------------------------+-------+-----------------------+----------------------------+--------------------------+---------+
|         201707|50270059011|    20331874213|           RK|                                -3|  366.5|                      1|                 44624209266|                        UZ|  ON_TIME|
|         201708|23386350591|    20179457076|           RB|                                 3|  80.08|                      1|                 44823969505|                        UN|remainder|
|         201710|42020201434|    19

In [18]:
final = spark.sql("""SELECT clearing_period,oib,sum(amount) as Amount,v.Interval from v \
group by clearing_period,oib,v.interval \
order by clearing_period,oib """).show(10)

+---------------+-----------+------+---------+
|clearing_period|        oib|Amount| Interval|
+---------------+-----------+------+---------+
|         201707|00668739878|210.34|remainder|
|         201707|02407755417| 100.0|remainder|
|         201707|03631133611|   0.0|  ON_TIME|
|         201707|04284881662|391.19|remainder|
|         201707|04566100589| 201.8|     4463|
|         201707|04760691261|451.79|remainder|
|         201707|05170776221| 98.19|  ON_TIME|
|         201707|05901038174|406.44|  ON_TIME|
|         201707|06299539916|228.87|remainder|
|         201707|07865774959|   0.0|  ON_TIME|
+---------------+-----------+------+---------+
only showing top 10 rows

